# Parts of Speech & Named Entity Recognition

![nlp](https://wrm5sysfkg-flywheel.netdna-ssl.com/wp-content/uploads/2019/01/NLP-Technology-in-Healthcare.jpg)

# 4.1.0 - Parts of Speech tagging

**What is it?** the tokenisation process can itself be difficult or can pose problems in many languages. That means overall we can consider that while it is possible to solve some problems starting only from the raw characters it is generally 'better' to use linguistic knowledge to add useful meaning and information.

**Why?** well, most words are rare and it's common for words that look completely different to mean almost the same thing. The same words rearranged in a different order can mean something wildly different. There is a lot of nuance to the way words are used and the order they are used in. This is why it's important to look at the `parts of speech` that a word appears in rather than the word or `token` itself. 

This process is exactly what Spacy was designed to do. You submit the raw text and a doc object that is annotated is what is returned. In the proces Spacy gives two tiers of annotations. We get coarse POS tags such as `nouns`, `verb` & `adjective` but also fine-grained tags such as `plural noun`, `past tense verb`, `superlative adjective`.  
